## 🧠 Model Splitting Across Nodes in Grid Using Thirdparty Inference Service like vLLM(with Ray)


- Author: Shridhar Kini (Profile)
- To Securely Run: `jupyter notebook password` to generate onetime password for secure access
- To Run: `jupyter notebook --allow-root  --port 9999 --ip=0.0.0.0 part-2_model_splitting_using_vllm.ipynb`
- To Clear Outputs: Use `jupyter nbconvert --clear-output --inplace part-2_model_splitting_using_vllm.ipynb`

This notebook provides an overview of Model Splitting across Nodes in AIOS using 3rd party systems like vLLM inference server using init containers concept. This Demo is for the users who has bigger models which cannot be fit in single GPU or Node. Splitting Models with the helper code from **init_container** using vLLM inference server is shown here.

**To  GET PORT MAPPING wrt to Service**([Doc](https://docs.aigr.id/installation/installation/#deploying-registry-services))

In [ ]:
# 🔧 Configuration Setup - Run this cell first to set up shared variables
import os

# Set configuration variables that will be available across all cells
GATEWAY_URL = "MANAGEMENTMASTER:30600"
CLUSTER_ID = "gcp-cluster-2"
GLOBAL_CLUSTER_METRICS_DB = "MANAGEMENTMASTER:30202"
GLOBAL_BLOCK_METRICS_DB = "MANAGEMENTMASTER:30201"
PARSER_URL = "MANAGEMENTMASTER:30501"
GLOBAL_CLUSTER_DB = "MANAGEMENTMASTER:30101"
GLOBAL_TASK_DB_SERVICE = "MANAGEMENTMASTER:30108"
COMPONENT_REGISTRY_SERVICE = "MANAGEMENTMASTER:30112"
GLOBAL_BLOCKDB_SERVICE = "MANAGEMENTMASTER:30100"
#SERVER_URL = "10.10.10.10:5000"  # For other API calls

# Set environment variables for bash cells
os.environ['GATEWAY_URL'] = GATEWAY_URL
os.environ['CLUSTER_ID'] = CLUSTER_ID
os.environ['GLOBAL_CLUSTER_METRICS_DB'] = GLOBAL_CLUSTER_METRICS_DB
os.environ['GLOBAL_BLOCK_METRICS_DB'] = GLOBAL_BLOCK_METRICS_DB
os.environ['PARSER_URL'] = PARSER_URL
os.environ['GLOBAL_CLUSTER_DB'] = GLOBAL_CLUSTER_DB
os.environ['GLOBAL_TASK_DB_SERVICE'] = GLOBAL_TASK_DB_SERVICE
os.environ['COMPONENT_REGISTRY_SERVICE'] = COMPONENT_REGISTRY_SERVICE
os.environ['GLOBAL_BLOCKDB_SERVICE'] = GLOBAL_BLOCKDB_SERVICE
#os.environ['SERVER_URL'] = SERVER_URL

print("✅ Configuration variables set:")
print(f"   • GATEWAY_URL: {GATEWAY_URL}")
print(f"   • CLUSTER_ID: {CLUSTER_ID}")
print(f"   • GLOBAL_CLUSTER_METRICS_DB: {GLOBAL_CLUSTER_METRICS_DB}")
print("\n📝 These variables are now available in both Python and bash cells!")
print("   - In Python: use GATEWAY_URL, CLUSTER_ID, GLOBAL_CLUSTER_METRICS_DB PARSER_URL GLOBAL_CLUSTER_DB GLOBAL_TASK_DB_SERVICE")
print("   - In bash: use $GATEWAY_URL, $CLUSTER_ID, $GLOBAL_CLUSTER_METRICS_DB $PARSER_URL $GLOBAL_CLUSTER_DB $GLOBAL_TASK_DB_SERVICE")
os.system('echo $GATEWAY_URL')
os.system('echo $CLUSTER_ID')
os.system('echo $GLOBAL_CLUSTER_METRICS_DB')
os.system('echo $PARSER_URL')
os.system('echo $GLOBAL_CLUSTER_DB')
os.system('echo $GLOBAL_TASK_DB_SERVICE')
os.system('echo $COMPONENT_REGISTRY_SERVICE')
os.system('echo $GLOBAL_BLOCKDB_SERVICE')
os.system('echo $GLOBAL_BLOCK_METRICS_DB')

### Code Sample for Splitting and Generating Response ([Code](Part-2/init_container))

##### Main Codes
- [init_container/main.py](Part-2/init_container/main.py) - For generating Deployment and Service File of vLLM Container and deploy it with LeaderWorker strategy
    - Leader and Worker:
        - One Leader pod which initializes Ray Cluster and vLLM Inference Server and N-1 workers pods which joins the Ray Cluster created by Leader pod
        - perfect for distributed applications like vLLM that use a central coordinator (the leader) and several computation nodes (the workers)
        - Leader Service is created with Http port(for vLLM inference) and Ray Port(for Coordinating).
    - Interacts with K8s cluster using K8s Python Client
- [init_container_sdk/sdk.py](Part-2/init_container/init_container_sdk/sdk.py) - SDK for interfacing init container with AIOS system.
- [vllm_client/main.py](Part-2/block/vllm-client/main.py) - Proxy Block which can interact with leader of vLLM replicas. 
    - It can hold chat history kind of logic
    - It can do inference with Splitted Model Rank-0
    - It can be extended to do other tasks like Metrics, Logging, etc
    - It can talk to any other API of vLLM if needed.

##### Important Changes from AIOS Flow
- [component.json](Part-2/component.json): `"componentMode": "aios"`   to `"componentMode": "third_party",
        "initContainer": {
            "image": "MANAGEMENTMASTER:31280/third-party/vllm"
        }`
- In [resource allocator policy](Part-2/policies/code/function.py):
            `if input_data['action'] == 'third_party_allocate':
                logging.info(f"parameters={self.parameters}")

                if 'third_party_allocation_data' in self.parameters:
                    return self.parameters['third_party_allocation_data']`
- init container :
    - creates vLLM container as initialization for model split using Ray backend. Post vllm container starts,  block container(instance of proxy block) will be created.
    - any other 3rd party/backend can be initialized like this.
    - command for leader:
        - `bash /vllm-workspace/examples/online_serving/multi-node-serving.sh leader --ray_cluster_size={self.replicas}`
            - script initializes distributed environment
            - leader: This argument tells the script to configure this pod as the head node of a Ray cluster. Ray is the underlying framework vLLM uses to coordinate tasks across multiple GPUs and nodes
            - --ray_cluster_size=$(self.replicas): This tells the Ray head node how many total nodes (pods) to expect in its cluster. $(self.replicas) value is equal to the size field (e.g., 4), so Ray knows to wait for 1 leader and 3 workers to join.
        - `python3 -m vllm.entrypoints.openai.api_server --port {self.api_port} --model {self.model} --max-model-len {self.max_model_len} --tensor-parallel-size {self.tensor_parallel} --pipeline-parallel-size {self.pipeline_parallel}`
            - After the Ray cluster is initialized, this command starts the main vLLM application. 
            - This process acts as the API endpoint for user requests. It receives prompts, forwards them to the distributed Ray cluster for processing by the LLM, and returns the generated text.
            - It listens on the specified --port.
            - --tensor-parallel-size and --pipeline-parallel-size are key vLLM arguments that define how the model is split across all available GPUs in the cluster for efficient, parallel processing.
    - command for worker:
        - `bash /vllm-workspace/examples/online_serving/multi-node-serving.sh worker --ray_address={ray_address}`
            - this command configures the pod as a worker node in the Ray cluster.
            - This argument tells the script to join an existing Ray cluster instead of starting a new one.
            - --ray_address=$(ray_address): This is the most important part. ray_address is leader_service_name.namespace.svc.cluster.local resolves to the internal IP address and port of the leader pod. This is how the worker knows exactly where to find and connect to the Ray head node, allowing it to join the cluster and start receiving computation tasks.
    - replicas - size vs tensor_parallel_size - pipeline_parallel_size:
        - Kubernetes (replicas, size) is responsible for providing the infrastructure (the pods and GPUs).
        - vLLM (--tensor-parallel-size, --pipeline-parallel-size) is responsible for using that infrastructure to run the model.
        - **Summary Analogy**
            - size: Determines how many workers are on your project team to complete one large task.
            - replicas: The number of independent project teams you are running simultaneously to handle more overall work.
            - --tensor-parallel-size & --pipeline-parallel-size: The internal work plan that dictates how your project team organizes itself to complete that task. The number of people required by the plan must match the team size.
            
            - total pods: replicas x size  (equal to Total number of GPUs available)
            - size: --tensor-parallel-size x --pipeline-parallel-size 
            - if replicas=3 and `--tensor-parallel-size=2 and --pipeline-parallel-size=4`
                - What you get is:
                    - Three completely separate vLLM deployments.
                    - Each deployment consists of 8 pods (1 leader, 7 workers).
                    - Each 8-pod deployment runs its own model instance using 2-way tensor and 4-way pipeline parallelism.
                    - You would have a total of 3 * 8 = 24 pods running in your cluster.
- Proxy Block for: 
    - Proxy Block can do inference with Splitted Model Rank-0
    - Proxy Block can hold chat history kind of logic


##### To Build the init container images

In [ ]:
%%bash
bash Part-2/init_container/build_docker.bash

##### Push the built docker to registry

In [ ]:
%%bash
docker push MANAGEMENTMASTER:31280/third-party/vllm:demo

##### To Build the block container image (Proxy Container)

In [ ]:
%%bash
bash Part-2/block/vllm-client/build_docker.bash

##### Push the built docker to registry

In [ ]:
%%bash
docker push MANAGEMENTMASTER:31280/example/vllm-client:demo

##### **Register the Blocks Component**

In [ ]:
%%bash
curl -X POST http://$COMPONENT_REGISTRY_SERVICE/api/registerComponent \
 -d @Part-2/component.json \
 -H "Content-Type: application/json" | json_pp

##### **UnRegister the Block Component**

In [ ]:
%%bash
curl -X POST http://$COMPONENT_REGISTRY_SERVICE/api/unregisterComponent \
  -H "Content-Type: application/json" \
  -d '{"uri":"model.vllm-runner-demo:1.0.0-stable"}' | json_pp

##### **Deploy the Proxy Block(With its init container)**

In [ ]:
%%bash
curl -X POST -d @Part-2/block.json \
 -H "Content-Type: application/json" \
  http://$PARSER_URL/api/createBlock | json_pp

##### To check the k8s pod(to be done in target cluster Controle Plane)

In [ ]:
kubectl get pods -n vllm-blocks #for vLLM Ray Cluster
kubectl get pods -n default  #for init container

##### To get the log of vLLM pods

In [ ]:
%%bash
kubectl logs -f vllm-0 -n vllm-blocks

##### To get the log of proxy Block

In [ ]:
kubectl get pods -n blocks

##### **Create Inference Server For The Cluster**
- Run this commands in your cluster node(like master node)
    - `kubectl create namespace inference-server`
    - `kubectl create -f inference_server/inference_server.yaml`

##### **Do the Inference**

In [ ]:
%%bash
curl -X POST  http://CLUSTER2_MASTER_IP:31504/v1/infer \
  -H "Content-Type: application/json" \
  -d '{
  "model": "vllm-block-demo-1",
  "session_id": "session-4",
  "seq_no": 23,
  "data": {
    "mode": "completions",
    "generation_config": {
                "max_tokens": 512,
                "top_k": 50,
                "top_p": 0.95,
                "temperature": 1.0
            },
    "message": "Give me code for adding two integers list element wise in c++",
    "system_message": "You are a helpful assistant that provides code examples."
  },
  "graph": {},
  "files": {},
  "selection_query": {
    
  }
}' | json_pp

#### Clean Up

##### Delete the Block

In [ ]:
%%bash
curl -X POST http://$GATEWAY_URL/controller/removeBlock/gcp-cluster-2 \
    -H "Content-Type: application/json" \
    -d '{"block_id": "vllm-block-demo-1"}'